In [34]:
from rdkit import Chem
import json

pdbcode = "5qj7"
pdbfile = open("../data/"+str(pdbcode)+".pdb").readlines()
non_ligs = json.load(open("non_ligs", "r"))
writer = Chem.rdmolfiles.SDWriter('../resulting_data/out.sdf')

def hets_and_cons(pdbfile):
    """
    Heteroatoms and connect files are pulled out from full pdb file
    
    param: pdb file in .readlines() format
    returns: lits of hetatomic information and connection information
    """
    hetatms  = []
    conects = []

    for line in pdbfile:
        if line.startswith("HETATM"):
            hetatms.append(line)
        if line.startswith("CONECT"):
            conects.append(line)
            
    return hetatms, conects

           
def remove_nonligands(hetatms, non_ligs):
    """
    Non-ligands such as solvents and ions are removed from the list of heteroatoms, 
    to ideally leave only the target ligand
    
    params: list of heteroatoms and their information, list of non-ligand small molecules 
        that could be in crystal structure
    retunrns: list of heteroatoms that are not contained in the non_ligs list
    """
    final_hets = []
    for line in hetatms:
        if str(line.split()[3]) not in non_ligs:
            final_hets.append(line)
    return final_hets

def find_ligand_names(final_hets):
    """
    Defining the individual ligands bound by type, chain bound and residue position
    
    param: list of ligand heteroatoms
    returns: list of individual ligands
    """
    ligands = []
    for i in range(len(final_hets)):
        line_split = final_hets[i].split()
        LIG_NAME = '_'.join(map(str, line_split[3:6]))
        if LIG_NAME not in ligands:
            ligands.append(LIG_NAME)
    return ligands

def create_pdb_for_ligand(pdbcode, ligand, final_hets, conects):
    """
    A pdb file is produced for an individual ligand, containing atomic and connection information
    
    params: vari pdb code, ligand definition, list of ligand heteroatoms and information, connection information
    returns: .pdb file for ligand
    """
    ligands_connections = open("../resulting_data/"+str(pdbcode)+"_"+str(ligand)+".pdb", "w+")
    individual_ligand = []
    individual_ligand_conect = []
    code  = ligand.split('_')
    for atom in final_hets:
        atom_info = atom.split()
        atom_number = atom_info[1]
        if code[0] == atom_info[3] and code[1] == atom_info[4] and code[2] == atom_info[5]:
            individual_ligand.append(atom)
            for conection in conects:
                if atom_number in conection and conection not in individual_ligand_conect:
                    individual_ligand_conect.append(conection)

    ligand_het_con = individual_ligand+individual_ligand_conect
    for line in ligand_het_con:
        ligands_connections.write(str(line))
    ligands_connections.close()

    return (Chem.rdmolfiles.MolFromPDBFile("../resulting_data/"+str(pdbcode)+"_"+str(ligand)+".pdb"))
            
def create_mol_file(ligand, mol_obj, pdbcode):
    """
    a .mol file is produced for an individual ligand
    
    params: ligand definition, pdb file, pdb code
    returns: .mol file for the ligand
    """
    return Chem.rdmolfiles.MolToMolFile(mol_obj, "../resulting_data/"+str(pdbcode)+"_"+str(ligand)+"_mol.mol")


def create_sd_file(mol_obj, writer):
    """
    a molecular object defined in the pdb file is used to produce a .sdf file
    
    params: pdb file for the molecule, SDWriter from rdkit
    returns: .sdf file with all input molecules from each time the function is called
    """
    writer.write(mol_obj)            
    
    
    
hetatms, conects = hets_and_cons(pdbfile)
final_hets = remove_nonligands(hetatms, non_ligs)
ligands = find_ligand_names(final_hets)

mol_1 = create_pdb_for_ligand(pdbcode, ligands[0], final_hets, conects)
mol_2 = create_pdb_for_ligand(pdbcode, ligands[1], final_hets, conects)
mol_3 = create_pdb_for_ligand(pdbcode, ligands[2], final_hets, conects)
mol_4 = create_pdb_for_ligand(pdbcode, ligands[3], final_hets, conects)

create_mol_file(ligands[0], mol_1, pdbcode)
create_mol_file(ligands[1], mol_2, pdbcode)
create_mol_file(ligands[2], mol_3, pdbcode)
create_mol_file(ligands[3], mol_4, pdbcode)

create_sd_file(mol_1, writer)
create_sd_file(mol_2, writer)
create_sd_file(mol_3, writer)
create_sd_file(mol_4, writer)
writer.close()